In [3]:
from collections import Counter
import itertools

import nltk
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from scipy import sparse
from scipy.sparse import linalg 
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
import pandas as pd


data = pd.read_csv("indoc.csv")
data.head(10)
data.shape

(1200, 5)

In [8]:
docs = data["Document"]

In [10]:

cl = [xs.replace('ইহাতে মন্তব্য প্রদান বন্ধ রয়েছে                                |    ', '') for xs in docs]
cl2 = [xs.replace('\xa0', '').replace('\u200c', '') for xs in cl]
cl3 = [xs.replace('?', u'।').replace('!', u'।').replace(';', u'।') for xs in cl]

sentTok =[]
wordTok = []
for doc in cl3:
    sentTok.append(doc.split('।'))
sentTok[0]

['স্নাতক পাস আবুল হাশেম (৩০) ও তাঁর ভাই উচ্চমাধ্যমিক পাস সুজন হোসেন (২৩) একসময় ঢাকায় গার্মেন্টসে চাকরি করতেন',
 ' তাঁদের আরেক ভাই অষ্টম শ্রেণি পাস ফরহাদ হোসেন (২৫) আগে নোয়াখালীতে গ্রামের বাড়িতে কৃষিকাজ করতেন',
 ' এখন তিন ভাইই পরিবার নিয়ে ঢাকার উত্তরার বহুতল ভবনে বাস করেন',
 ' ভাড়া দেন ৫০ হাজার টাকা',
 ' গত শনিবার তিন ভাইয়ের দুজন সুজন ও ফরহাদকে ২০ হাজার পিছ ইয়াবাসহ উত্তরার সেই ফ্ল্যাট থেকে গ্রেপ্তার করেছে ঢাকা মহানগর গোয়েন্দা পুলিশ (ডিবি)',
 ' এই ইয়াবার আনুমানিক মূল্য ৩০ লাখ টাকা',
 ' আর তিন ভাইয়ের ফ্ল্যাট থেকে জব্দ করা হয়েছে আরও নগদ ২ লাখ ৫৭ হাজার টাকা',
 ' রোববার সুজন ও ফরহাদকে ঢাকার আদালতে হাজির করে ডিবি এক প্রতিবেদন দিয়ে বলছে, হাশেম, সুজন ও ফরহাদ তিন ভাই কয়েক বছর ধরে কক্সবাজার থেকে ইয়াবার চালান ঢাকায় এনে জমজমাট ব্যবসা করছেন',
 ' তাঁদের সঙ্গে কারা জড়িত, তা খুঁজে বের করতে দুই ভাইকে তিন দিন জিজ্ঞাসাবাদের অনুমতি দিয়েছেন আদালত',
 ' তিনজনের বাবা ছিলেন হকারমাদক ব্যবসার অভিযোগ থাকা এই তিনজনের বাবার নাম বিল্লাল হোসেন',
 ' তাঁর গ্রামের বাড়ি নোয়াখালী',
 ' অনেক আগে থেকে তিনি ঢাকায় থাকেন',
 ' রা

In [57]:


import re
pnList = ['.', ',', ':', '(', ')', '[', ']', '{', '}', '`', '*', '&', '‘', '’', '–']
cl4 = [xs.replace(u'।'," ") for xs in cl3]

cl5=[[] for x in range(len(cl4))]
for i in range(len(cl4)):
#     cl5[i] = cl4[i].replace("(","").replace(")","").replace(",","").replace("+","").replace(")","")
    cl5[i] = re.sub("[০১২৩৪৫৬৭৮৯]",'',cl4[i])
    cl5[i] = re.sub("[,./<>)(}{_*&^%$#@~`]",'',cl5[i])

for doc in cl5:
    wordTok.append(doc.split())


#remove stopword
stRem = [[] for i in range(len(wordTok))]

def remStWords(punctuF):
    stWord = open("./stopwords-bn.txt", encoding='utf8')
    stWordFile = [i.rstrip() for i in stWord]
    filtStWord = [w for w in punctuF if w not in stWordFile]

    return filtStWord
for i in range(len(wordTok)):
    stRem[i] = remStWords(wordTok[i])
    

print(stRem[1:4])
headlines =stRem[:]

[['ঢাকার', 'কামরাঙ্গীরচরের', 'নবাবচরে', 'বুড়িগঙ্গা', 'নদী', 'দখল', 'স্থায়ী', 'রূপ', 'নিচ্ছে', 'নদীর', 'সীমানা', 'নির্ধারণ', 'বিষয়ে', 'জেলা', 'প্রশাসনের', 'প্রতিবেদনের', 'কারণে', 'অবস্থার', 'সৃষ্টি', 'রয়েছে', 'প্রভাবশালী', 'ব্যক্তিদের', 'চাপ', 'অভ্যন্তরীণ', 'নৌপরিবহন', 'কর্তৃপক্ষের', 'বিআইডব্লিউটিএ', 'উদ্যোগের', 'অভাব', 'বিআইডব্লিউটিএ', 'জেলা', 'প্রশাসন', 'সূত্রে', 'নৌপরিবহনমন্ত্রী', 'শাজাহান', 'খান', 'গত', 'বছরের', 'এপ্রিলে', 'বুড়িগঙ্গার', 'কামরাঙ্গীরচর', 'এলাকা', 'পরিদর্শন', 'সময়', 'নদীর', 'জায়গায়', 'অবৈধ', 'স্থাপনা', 'গড়ে', 'উঠেছে', 'সেগুলো', 'ভাঙার', 'ঘোষণা', 'এরপর', 'জুলাই', 'বিআইডব্লিউটিএ', 'অভিযান', 'চালিয়ে', 'তিনটি', 'স্থাপনার', 'বাইরের', 'দেয়ালের', 'সামান্য', 'অংশ', 'ভেঙে', 'দেয়', 'সাংসদ', 'হাজি', 'সেলিমের', 'মালিকানাধীন', 'স্থাপনার', 'সীমানাদেয়াল', 'ভাঙা', 'এরপরই', 'নবাবচরে', 'নদীতীরের', 'ভবনমালিকেরা', 'জেলা', 'প্রশাসনের', 'সীমানা', 'নির্ধারণের', 'আবেদন', 'জানালে', 'কমিটি', 'কমিটি', 'সিএস', 'ক্যাডেস্টাইল', 'সার্ভে', 'আরএস', 'রিভাইস', 'সার্ভে', 'রেকর্ড', 'অনুযায়ী', 'নদীর', 'সীম

In [148]:
unigram_counts = Counter()
for ii, headline in enumerate(headlines):
    if ii % 200000 == 0:
        print(f'finished {ii/len(headlines):.2%} of headlines')
    for token in headline:
        unigram_counts[token] += 1

tok2indx = {tok: indx for indx, tok in enumerate(unigram_counts.keys())}
indx2tok = {indx: tok for tok,indx in tok2indx.items()}
print('done')
print('vocabulary size: {}'.format(len(unigram_counts)))
print('most common: {}'.format(unigram_counts.most_common(60)))

finished 0.00% of headlines
done
vocabulary size: 46450
most common: [('হয়েছে', 9185), ('এক', 8605), ('কথা', 7590), ('হয়ে', 7460), ('’', 6630), ('দিয়ে', 6465), ('সময়', 5855), ('যায়', 4875), ('বছর', 4395), ('একটা', 4230), ('হিসেবে', 4130), ('ভালো', 3990), ('গত', 3855), ('বাংলাদেশ', 3710), ('বাংলাদেশের', 3520), ('শেষ', 3145), ('বড়', 3120), ('দেশের', 2820), ('একজন', 2540), (':', 2525), ('নাম', 2460), ('ওপর', 2450), ('সালে', 2405), ('ছবি', 2340), ('মানুষ', 2265), ('ছবির', 2220), ('ছাড়া', 2185), ('হওয়ার', 2155), ('পুলিশ', 2125), ('বছরের', 2045), ('দলের', 2035), ('রয়েছে', 1980), ('প্রেসিডেন্ট', 1930), ('দেশে', 1900), ('টাকা', 1845), ('গান', 1845), ('জানান', 1840), ('তিন', 1815), ('মা', 1800), ('সবাই', 1790), ('দল', 1790), ('বিষয়ে', 1780), ('তৈরি', 1775), ('মানুষের', 1775), ('জাতীয়', 1750), ('বিরুদ্ধে', 1745), ('অভিযোগ', 1670), ('দেওয়ার', 1670), ('প্রধানমন্ত্রী', 1640), ('পাওয়া', 1635), ('পড়ে', 1630), ('ট্রাম্প', 1625), ('অংশ', 1620), ('হাতে', 1605), ('হয়নি', 1595), ('এখনো', 1590), ('লাখ', 15

In [61]:
# note add dynammic window hyperparameter

# note we store the token vocab indices in the skipgram counter
# note we use Levy, Goldberg, Dagan notation (word, context) as opposed to (focus, context)
back_window = 2
front_window = 2
skipgram_counts = Counter()
for iheadline, headline in enumerate(headlines):
    tokens = [tok2indx[tok] for tok in headline]
    for ii_word, word in enumerate(tokens):
        ii_context_min = max(0, ii_word - back_window)
        ii_context_max = min(len(headline) - 1, ii_word + front_window)
        ii_contexts = [
            ii for ii in range(ii_context_min, ii_context_max + 1) 
            if ii != ii_word]
        for ii_context in ii_contexts:
            skipgram = (tokens[ii_word], tokens[ii_context])
            skipgram_counts[skipgram] += 1    
    if iheadline % 200000 == 0:
        print(f'finished {iheadline/len(headlines):.2%} of headlines')
        
print('done')
print('number of skipgrams: {}'.format(len(skipgram_counts)))
most_common = [
    (indx2tok[sg[0][0]], indx2tok[sg[0][1]], sg[1]) 
    for sg in skipgram_counts.most_common(10)]
print('most common: {}'.format(most_common))


finished 0.00% of headlines
done
number of skipgrams: 1003148
most common: [(':', ':', 760), ('আন্তর্জাতিক', 'মাতৃভাষা', 650), ('মাতৃভাষা', 'আন্তর্জাতিক', 650), ('খবরে', 'হয়েছে', 620), ('হয়েছে', 'খবরে', 620), ('মধ্য', 'দিয়ে', 575), ('দিয়ে', 'মধ্য', 575), ('হয়ে', 'যায়', 520), ('যায়', 'হয়ে', 520), ('লাখ', 'টাকা', 505)]


In [62]:
row_indxs = []
col_indxs = []
dat_values = []
ii = 0
for (tok1, tok2), sg_count in skipgram_counts.items():
    ii += 1
    if ii % 1000000 == 0:
        print(f'finished {ii/len(skipgram_counts):.2%} of skipgrams')    
    row_indxs.append(tok1)
    col_indxs.append(tok2)
    dat_values.append(sg_count)
wwcnt_mat = sparse.csr_matrix((dat_values, (row_indxs, col_indxs)))
print('done')

finished 99.69% of skipgrams
done


In [63]:
def ww_sim(word, mat, topn=10):
    """Calculate topn most similar words to word"""
    indx = tok2indx[word]
    if isinstance(mat, sparse.csr_matrix):
        v1 = mat.getrow(indx)
    else:
        v1 = mat[indx:indx+1, :]
    sims = cosine_similarity(mat, v1).flatten()
    sindxs = np.argsort(-sims)
    sim_word_scores = [(indx2tok[sindx], sims[sindx]) for sindx in sindxs[0:topn]]
    return sim_word_scores

In [64]:
ww_sim('নদীর', wwcnt_mat)

[('নদীর', 0.9999999999999937),
 ('নদী', 0.3182332385590676),
 ('ওয়াইকাটো', 0.31158147653600443),
 ('হাঁটবে', 0.29137624733120093),
 ('রাকিব', 0.2788318924368383),
 ('টেমস', 0.2742364680764244),
 ('মুন্সিহাটি', 0.2742364680764244),
 ('বলেশ্বর', 0.26663239106004577),
 ('সুরমা', 0.2545127369209528),
 ('একটা', 0.25051437895219125)]

In [65]:
# normalize each row using L2 norm
wwcnt_norm_mat = normalize(wwcnt_mat, norm='l2', axis=1)

In [66]:
# demonstrate normalization
row = wwcnt_mat.getrow(10).toarray().flatten()
print(np.sqrt((row*row).sum()))

row = wwcnt_norm_mat.getrow(10).toarray().flatten()
print(np.sqrt((row*row).sum()))

15.811388300841896
1.0


In [67]:
ww_sim('নদীর', wwcnt_mat)

[('নদীর', 0.9999999999999937),
 ('নদী', 0.3182332385590676),
 ('ওয়াইকাটো', 0.31158147653600443),
 ('হাঁটবে', 0.29137624733120093),
 ('রাকিব', 0.2788318924368383),
 ('টেমস', 0.2742364680764244),
 ('মুন্সিহাটি', 0.2742364680764244),
 ('বলেশ্বর', 0.26663239106004577),
 ('সুরমা', 0.2545127369209528),
 ('একটা', 0.25051437895219125)]

In [81]:
num_skipgrams = wwcnt_mat.sum()
assert(sum(skipgram_counts.values())==num_skipgrams)

# for creating sparce matrices
row_indxs = []
col_indxs = []

pmi_dat_values = []    # pointwise mutual information
ppmi_dat_values = []   # positive pointwise mutial information
spmi_dat_values = []   # smoothed pointwise mutual information
sppmi_dat_values = []  # smoothed positive pointwise mutual information

# reusable quantities

# sum_over_rows[ii] = sum_over_words[ii] = wwcnt_mat.getcol(ii).sum()
sum_over_words = np.array(wwcnt_mat.sum(axis=0)).flatten()
# sum_over_cols[ii] = sum_over_contexts[ii] = wwcnt_mat.getrow(ii).sum()
sum_over_contexts = np.array(wwcnt_mat.sum(axis=1)).flatten()

# smoothing
alpha = 0.75
sum_over_words_alpha = sum_over_words**alpha
nca_denom = np.sum(sum_over_words_alpha)



ii = 0
for (tok_word, tok_context), sg_count in skipgram_counts.items():

    ii += 1
    if ii % 1000000 == 0:
        print(f'finished {ii/len(skipgram_counts):.2%} of skipgrams')

    # here we have the following correspondance with Levy, Goldberg, Dagan
    #========================================================================
    #   num_skipgrams = |D|
    #   nwc = sg_count = #(w,c)
    #   Pwc = nwc / num_skipgrams = #(w,c) / |D|
    #   nw = sum_over_cols[tok_word]    = sum_over_contexts[tok_word] = #(w)
    #   Pw = nw / num_skipgrams = #(w) / |D|
    #   nc = sum_over_rows[tok_context] = sum_over_words[tok_context] = #(c)
    #   Pc = nc / num_skipgrams = #(c) / |D|
    #
    #   nca = sum_over_rows[tok_context]^alpha = sum_over_words[tok_context]^alpha = #(c)^alpha
    #   nca_denom = sum_{tok_content}( sum_over_words[tok_content]^alpha )
    
    nwc = sg_count
    Pwc = nwc / num_skipgrams
    nw = sum_over_contexts[tok_word]
    Pw = nw / num_skipgrams
    nc = sum_over_words[tok_context]
    Pc = nc / num_skipgrams
    
    nca = sum_over_words_alpha[tok_context]
    Pca = nca / nca_denom
    
    # note 
    # pmi = log {#(w,c) |D| / [#(w) #(c)]} 
    #     = log {nwc * num_skipgrams / [nw nc]}
    #     = log {P(w,c) / [P(w) P(c)]} 
    #     = log {Pwc / [Pw Pc]}
    pmi = np.log2(Pwc/(Pw*Pc))   
    ppmi = max(pmi, 0)
    spmi = np.log2(Pwc/(Pw*Pca))
    sppmi = max(spmi, 0)
    
    row_indxs.append(tok_word)
    col_indxs.append(tok_context)
    pmi_dat_values.append(pmi)
    ppmi_dat_values.append(ppmi)
    spmi_dat_values.append(spmi)
    sppmi_dat_values.append(sppmi)
        
pmi_mat = sparse.csr_matrix((pmi_dat_values, (row_indxs, col_indxs)))
ppmi_mat = sparse.csr_matrix((ppmi_dat_values, (row_indxs, col_indxs)))
spmi_mat = sparse.csr_matrix((spmi_dat_values, (row_indxs, col_indxs)))
sppmi_mat = sparse.csr_matrix((sppmi_dat_values, (row_indxs, col_indxs)))

print('done')

print(pmi_mat[2,:600])

finished 99.69% of skipgrams
done
  (0, 0)	7.874433025835088
  (0, 1)	4.426620490464623
  (0, 3)	9.22493027291922
  (0, 4)	4.669574234283388
  (0, 7)	5.468707560071932
  (0, 16)	7.180536153560768
  (0, 18)	4.067078103777483
  (0, 57)	0.6262263801393225
  (0, 102)	5.641772269044812
  (0, 105)	4.215381545946963
  (0, 107)	3.1937115423085003
  (0, 116)	4.764679173975375
  (0, 117)	4.289470525113931
  (0, 118)	8.261456148944335
  (0, 132)	3.474055833262376
  (0, 134)	6.7139683536418415
  (0, 141)	10.961895867085428
  (0, 244)	4.77207130820541
  (0, 266)	0.39846123137522904
  (0, 317)	3.486162436119029
  (0, 321)	0.6575445454221883
  (0, 451)	5.303684384333632
  (0, 456)	4.388248679592104
  (0, 537)	4.7139683536418415
  (0, 564)	7.283823961972789
  (0, 567)	5.99611158242334
  (0, 572)	3.855987358514269


In [69]:
ww_sim('নদীর', pmi_mat)

[('নদীর', 0.9999999999999969),
 ('পাড়ে', 0.19705428427709587),
 ('নদী', 0.16331044649420565),
 ('রাকিব', 0.13575929707086798),
 ('উন্মুখ', 0.12146604588243068),
 ('লাগোয়া', 0.11953663758266395),
 ('সীমানা', 0.11680274915480077),
 ('সিথানের', 0.10477186350305927),
 ('পাড়', 0.10356093073450118),
 ('মোবাইলটা', 0.1031384276500718)]

In [70]:
ww_sim('নদীর',  ppmi_mat)

[('নদীর', 0.9999999999999969),
 ('পাড়ে', 0.1970670268332455),
 ('নদী', 0.1631518628864339),
 ('রাকিব', 0.13577561593844395),
 ('উন্মুখ', 0.12147390050947217),
 ('লাগোয়া', 0.1195443674441177),
 ('সীমানা', 0.11717992805555133),
 ('সিথানের', 0.10477863859733585),
 ('পাড়', 0.10356762752355922),
 ('মোবাইলটা', 0.10314509711788028)]

In [71]:
ww_sim('নদীর', spmi_mat)

[('নদীর', 1.0),
 ('পাড়ে', 0.19478339096496086),
 ('নদী', 0.1753261818262632),
 ('রাকিব', 0.1533154034403254),
 ('উন্মুখ', 0.12725691269706918),
 ('সীমানা', 0.12491585508804463),
 ('সিথানের', 0.11261877207680861),
 ('লাগোয়া', 0.11213501556542993),
 ('মোবাইলটা', 0.1110602360882953),
 ('কোনোমতেই', 0.10535949558044788)]

In [72]:
ww_sim('নদীর', sppmi_mat)

[('নদীর', 1.0),
 ('পাড়ে', 0.19478339096496086),
 ('নদী', 0.17533930244726986),
 ('রাকিব', 0.1533154034403254),
 ('উন্মুখ', 0.12725691269706918),
 ('সীমানা', 0.12491585508804463),
 ('সিথানের', 0.11261877207680861),
 ('লাগোয়া', 0.11213501556542993),
 ('মোবাইলটা', 0.1110602360882953),
 ('কোনোমতেই', 0.10535949558044788)]

In [73]:
pmi_use = ppmi_mat
embedding_size = 50
uu, ss, vv = linalg.svds(pmi_use, embedding_size) 

In [74]:
print('vocab size: {}'.format(len(unigram_counts)))
print('embedding size: {}'.format(embedding_size))
print('uu.shape: {}'.format(uu.shape))
print('ss.shape: {}'.format(ss.shape))
print('vv.shape: {}'.format(vv.shape))

vocab size: 46450
embedding size: 50
uu.shape: (46450, 50)
ss.shape: (50,)
vv.shape: (50, 46450)


In [75]:
unorm = uu / np.sqrt(np.sum(uu*uu, axis=1, keepdims=True))
vnorm = vv / np.sqrt(np.sum(vv*vv, axis=0, keepdims=True))
#word_vecs = unorm
#word_vecs = vnorm.T
word_vecs = uu + vv.T
word_vecs_norm = word_vecs / np.sqrt(np.sum(word_vecs*word_vecs, axis=1, keepdims=True))

In [76]:
def word_sim_report(word, sim_mat):
    sim_word_scores = ww_sim(word, word_vecs)
    for sim_word, sim_score in sim_word_scores:
        print(sim_word, sim_score)
        word_headlines = [hl for hl in headlines if sim_word in hl and word in hl][0:5]
        for headline in word_headlines:
            print(f'    {headline}')

In [77]:
word = 'বুড়িগঙ্গা'
word_sim_report(word, word_vecs)

বুড়িগঙ্গা 0.9999999999999998
    ['ঢাকার', 'কামরাঙ্গীরচরের', 'নবাবচরে', 'বুড়িগঙ্গা', 'নদী', 'দখল', 'স্থায়ী', 'রূপ', 'নিচ্ছে', 'নদীর', 'সীমানা', 'নির্ধারণ', 'বিষয়ে', 'জেলা', 'প্রশাসনের', 'প্রতিবেদনের', 'কারণে', 'অবস্থার', 'সৃষ্টি', 'রয়েছে', 'প্রভাবশালী', 'ব্যক্তিদের', 'চাপ', 'অভ্যন্তরীণ', 'নৌপরিবহন', 'কর্তৃপক্ষের', 'বিআইডব্লিউটিএ', 'উদ্যোগের', 'অভাব', 'বিআইডব্লিউটিএ', 'জেলা', 'প্রশাসন', 'সূত্রে', 'নৌপরিবহনমন্ত্রী', 'শাজাহান', 'খান', 'গত', 'বছরের', 'এপ্রিলে', 'বুড়িগঙ্গার', 'কামরাঙ্গীরচর', 'এলাকা', 'পরিদর্শন', 'সময়', 'নদীর', 'জায়গায়', 'অবৈধ', 'স্থাপনা', 'গড়ে', 'উঠেছে', 'সেগুলো', 'ভাঙার', 'ঘোষণা', 'এরপর', 'জুলাই', 'বিআইডব্লিউটিএ', 'অভিযান', 'চালিয়ে', 'তিনটি', 'স্থাপনার', 'বাইরের', 'দেয়ালের', 'সামান্য', 'অংশ', 'ভেঙে', 'দেয়', 'সাংসদ', 'হাজি', 'সেলিমের', 'মালিকানাধীন', 'স্থাপনার', 'সীমানাদেয়াল', 'ভাঙা', 'এরপরই', 'নবাবচরে', 'নদীতীরের', 'ভবনমালিকেরা', 'জেলা', 'প্রশাসনের', 'সীমানা', 'নির্ধারণের', 'আবেদন', 'জানালে', 'কমিটি', 'কমিটি', 'সিএস', 'ক্যাডেস্টাইল', 'সার্ভে', 'আরএস', 'রিভাইস', 'সার্ভে', '

কেরানীগঞ্জের 0.7532200944095139
    ['রাজধানীর', 'সদরঘাটে', 'বুড়িগঙ্গা', 'নদীতে', 'নৌকাডুবির', 'ঘটনায়', 'নিখোঁজ', 'শম্পা', 'বেগম', 'সানজিদা', 'আক্তারের', 'লাশ', 'উদ্ধার', 'হয়েছে', 'সোমবার', 'রাত', 'টার', 'লাশ', 'উদ্ধার', 'ফায়ার', 'সার্ভিস', 'বিআইডব্লিউটিএর', 'ডুবুরিরা', 'সোমবার', 'সন্ধ্যা', 'সাড়ে', 'ছয়টার', 'নৌকাডুবির', 'ঘটনা', 'ঘটে', 'ঢাকা', 'নৌবন্দরের', 'উপপরিচালক', 'মিজানুর', 'রহমান', 'রাত', 'দশটার', 'মাঝনদী', 'কিশোরীর', 'লাশ', 'উদ্ধার', 'হয়েছে', 'কিশোরী', 'শম্পার', 'মা', 'শাহিনুর', 'বেগম', 'সোমবার', 'সন্ধ্যা', 'সাড়ে', 'টার', 'সদরঘাটের', 'লালকুঠি', 'খেয়াঘাট', 'নৌকাযোগে', 'কেরানীগঞ্জের', 'তেলঘাটে', 'যাওয়ার', 'সময়', 'বাংলাদেশ', 'অভ্যন্তরীণ', 'নৌপরিবহন', 'করপোরেশনের', 'বিআইডব্লিউটিসি', 'যাত্রীবাহী', 'জাহাজ', 'এম', 'ভি', 'বাঙালির', 'প্রচণ্ড', 'ঢেউয়ের', 'কারণে', 'নৌকাটি', 'মাঝ', 'নদীতে', 'ডুবে', 'যায়', 'সময়', 'ছয়জন', 'সাঁতার', 'কেটে', 'তীরে', 'উঠতে', 'পারলেও', 'শম্পা', 'অপরযাত্রী', 'ডুবে', 'যায়', 'কথা', 'নৌকা', 'ডুবির', 'ঘটনায়', 'বেঁচে', 'অপরযাত্রী', 'জিয়াউল', 'ইসলাম', 'বিআইডব্লিউটিসির',

সদরঘাটে 0.7412470930352129
    ['রাজধানীর', 'সদরঘাটে', 'বুড়িগঙ্গা', 'নদীতে', 'নৌকাডুবির', 'ঘটনায়', 'নিখোঁজ', 'শম্পা', 'বেগম', 'সানজিদা', 'আক্তারের', 'লাশ', 'উদ্ধার', 'হয়েছে', 'সোমবার', 'রাত', 'টার', 'লাশ', 'উদ্ধার', 'ফায়ার', 'সার্ভিস', 'বিআইডব্লিউটিএর', 'ডুবুরিরা', 'সোমবার', 'সন্ধ্যা', 'সাড়ে', 'ছয়টার', 'নৌকাডুবির', 'ঘটনা', 'ঘটে', 'ঢাকা', 'নৌবন্দরের', 'উপপরিচালক', 'মিজানুর', 'রহমান', 'রাত', 'দশটার', 'মাঝনদী', 'কিশোরীর', 'লাশ', 'উদ্ধার', 'হয়েছে', 'কিশোরী', 'শম্পার', 'মা', 'শাহিনুর', 'বেগম', 'সোমবার', 'সন্ধ্যা', 'সাড়ে', 'টার', 'সদরঘাটের', 'লালকুঠি', 'খেয়াঘাট', 'নৌকাযোগে', 'কেরানীগঞ্জের', 'তেলঘাটে', 'যাওয়ার', 'সময়', 'বাংলাদেশ', 'অভ্যন্তরীণ', 'নৌপরিবহন', 'করপোরেশনের', 'বিআইডব্লিউটিসি', 'যাত্রীবাহী', 'জাহাজ', 'এম', 'ভি', 'বাঙালির', 'প্রচণ্ড', 'ঢেউয়ের', 'কারণে', 'নৌকাটি', 'মাঝ', 'নদীতে', 'ডুবে', 'যায়', 'সময়', 'ছয়জন', 'সাঁতার', 'কেটে', 'তীরে', 'উঠতে', 'পারলেও', 'শম্পা', 'অপরযাত্রী', 'ডুবে', 'যায়', 'কথা', 'নৌকা', 'ডুবির', 'ঘটনায়', 'বেঁচে', 'অপরযাত্রী', 'জিয়াউল', 'ইসলাম', 'বিআইডব্লিউটিসির', 'যাত

In [135]:
v = pmi_mat[:4,:4]
print(v)



  (0, 1)	9.044790036575698
  (0, 2)	7.874433025835088
  (1, 0)	9.044790036575698
  (1, 1)	4.275049406317873
  (1, 2)	4.426620490464623
  (1, 3)	6.4883966880513135
  (2, 0)	7.874433025835088
  (2, 1)	4.426620490464623
  (2, 3)	9.22493027291922
  (3, 1)	6.4883966880513135
  (3, 2)	9.22493027291922


In [144]:
import sklearn.feature_selection

y = sklearn.feature_selection.mutual_info_classif(pmi_mat[:1200,:], list(data['class']), discrete_features='auto', n_neighbors=3, copy=True, random_state=None)

In [146]:
y

0.013462367562590928

In [126]:
from sklearn.model_selection import train_test_split 

from sklearn import svm
from sklearn import preprocessing


dataX = v
dataY = data['class']
print(len(dataX))

trainX, testX, trainY, testY = train_test_split(dataX, dataY, test_size = 0.20, random_state = 0,stratify=dataY)

1200


In [127]:

#Import model for NB
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.metrics import accuracy_score
# create classifier object gor model
model=GaussianNB()

# train the model with fit function
model.fit(trainX, trainY)
   
# make predictions on training data
predictions_train = model.predict(trainX)
print('\nTraining Accuracy using NB :',accuracy_score(trainY, predictions_train))

# make predictions on test data
predictions_test = model.predict(testX)
print('\nTest Accuracy using NB :',accuracy_score(testY, predictions_test))


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print('\nConfusion matrix:\n',confusion_matrix(trainY,predictions_train))
print(classification_report(trainY,predictions_train))  

print('\nConfusion matrix:\n',confusion_matrix(testY,predictions_test))
print(classification_report(testY,predictions_test)) 



Training Accuracy using NB : 0.9614583333333333

Test Accuracy using NB : 0.3625

Confusion matrix:
 [[160   0   0   0   0   0]
 [  1 159   0   0   0   0]
 [  2   2 153   2   0   1]
 [  7   0   0 149   0   4]
 [  5   2   1   1 149   2]
 [  7   0   0   0   0 153]]
                 precision    recall  f1-score   support

           ['']       0.88      1.00      0.94       160
['আন্তর্জাতিক']       0.98      0.99      0.98       160
       ['খেলা']       0.99      0.96      0.97       160
  ['দূর পরবাস']       0.98      0.93      0.96       160
   ['বাংলাদেশ']       1.00      0.93      0.96       160
     ['বিনোদন']       0.96      0.96      0.96       160

       accuracy                           0.96       960
      macro avg       0.96      0.96      0.96       960
   weighted avg       0.96      0.96      0.96       960


Confusion matrix:
 [[10  0  0  2 25  3]
 [ 0 12  1  1 26  0]
 [ 0  2 12  0 25  1]
 [ 2  0  1  8 27  2]
 [ 0  0  1  1 36  2]
 [ 0  3  0  4 24  9]]
               

In [124]:

#Import model for NB
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.metrics import accuracy_score
# create classifier object gor model
model=svm.SVC(kernel='linear', C=1)

# train the model with fit function
model.fit(trainX, trainY)
   
# make predictions on training data
predictions_train = model.predict(trainX)
print('\nTraining Accuracy using NB :',accuracy_score(trainY, predictions_train))

# make predictions on test data
predictions_test = model.predict(testX)
print('\nTest Accuracy using NB :',accuracy_score(testY, predictions_test))


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print('\nConfusion matrix:\n',confusion_matrix(trainY,predictions_train))
print(classification_report(trainY,predictions_train))  

print('\nConfusion matrix:\n',confusion_matrix(testY,predictions_test))
print(classification_report(testY,predictions_test)) 



Training Accuracy using NB : 1.0

Test Accuracy using NB : 0.6375

Confusion matrix:
 [[160   0   0   0   0   0]
 [  0 160   0   0   0   0]
 [  0   0 160   0   0   0]
 [  0   0   0 160   0   0]
 [  0   0   0   0 160   0]
 [  0   0   0   0   0 160]]
                 precision    recall  f1-score   support

           ['']       1.00      1.00      1.00       160
['আন্তর্জাতিক']       1.00      1.00      1.00       160
       ['খেলা']       1.00      1.00      1.00       160
  ['দূর পরবাস']       1.00      1.00      1.00       160
   ['বাংলাদেশ']       1.00      1.00      1.00       160
     ['বিনোদন']       1.00      1.00      1.00       160

       accuracy                           1.00       960
      macro avg       1.00      1.00      1.00       960
   weighted avg       1.00      1.00      1.00       960


Confusion matrix:
 [[28  4  4  0  1  3]
 [ 0 25  4  8  1  2]
 [ 1  6 23  3  2  5]
 [ 3  0  1 27  1  8]
 [ 4  3  1  3 28  1]
 [ 4  4  3  5  2 22]]
                 precision    